In [1]:
import sys
sys.path.append('/home/sean/Documents/Work/Level 4/Level-4-Masters-Project/')
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras import models 
import os
from keras.regularizers import l2
import matplotlib
import Hyperparam_Testing.Testing_Notebooks.Common_Functions as cf 
import Hyperparam_Testing.Testing_Notebooks.chirallag as cL

matplotlib.rcParams.update({'legend.fontsize': 16})
matplotlib.rcParams.update({'font.size': 16})
from keras import backend as K
import os


In [3]:

##Load Training Data
treaining_data_array = np.loadtxt('/home/sean/Documents/Work/Level 4/Level-4-Masters-Project/Batch_size_gradient_variance/Weights_Data/training_data.csv', delimiter = ',')
training_data = ([treaining_data_array[:-1000,:16],treaining_data_array[:-1000,16]],[treaining_data_array[-1000:,:16],treaining_data_array[-1000:,16]])

In [5]:


#Hyperparam Which is edited
batch_size = 256
# Default hyperparams for network 
epochs = 100
nodes_per_layer = 300
optimizer = keras.optimizers.Adam(beta_1 = 0.9,beta_2 = 0.98)

#split_training_data
train_x, train_y = training_data[0]
val_x,val_y = training_data[1]

#Dictionary to append weights to 
weights_dict = {}
#Callback to record weights

class GetWeights(tf.keras.callbacks.Callback):
    # Keras callback which collects values of weights and biases at each epoch
    def __init__(self):
        super(GetWeights, self).__init__()
        #weights_dict = {}

    def on_epoch_end(self, epoch, logs=None):
        # this function runs at the end of each epoch

        # loop over each layer and get weights and biases
        for layer_i in range(len(self.model.layers)):
            w = self.model.layers[layer_i].get_weights()[0]
            b = self.model.layers[layer_i].get_weights()[1]
            #print('Layer %s has weights of shape %s and biases of shape %s' %(
                #layer_i, np.shape(w), np.shape(b)))

            # save all weights and biases inside a dictionary
            if epoch == 0:
                # create array to hold weights and biases
                weights_dict['w_'+str(layer_i+1)] = w
                weights_dict['b_'+str(layer_i+1)] = b
            else:
                # append new weights to previously-created weights array
                weights_dict['w_'+str(layer_i+1)] = np.dstack(
                    (weights_dict['w_'+str(layer_i+1)], w))
                # append new weights to previously-created weights array
                weights_dict['b_'+str(layer_i+1)] = np.dstack(
                    (weights_dict['b_'+str(layer_i+1)], b))

In [6]:

K.clear_session()
#Compile network
model = models.Sequential()
model.add(layers.Dense(nodes_per_layer, activation ='relu', kernel_initializer= 'random_normal', input_shape = [train_x.shape[1]]))
model.add(layers.Dense(nodes_per_layer, activation ='relu', kernel_initializer= 'random_normal'))
model.add(layers.Dense(nodes_per_layer, activation ='relu', kernel_initializer= 'random_normal'))
model.add(layers.Dense(nodes_per_layer, activation ='relu', kernel_initializer= 'random_normal'))
model.add(layers.Dense(nodes_per_layer, activation ='relu', kernel_initializer= 'random_normal'))
model.add(layers.Dense(1))
model.compile(optimizer = optimizer,loss = 'mape', metrics = ['mean_absolute_percentage_error'])
#model.summary()

# Train Network 
model.fit(train_x,train_y,validation_data=(val_x,val_y),batch_size= batch_size , epochs = epochs, callbacks= GetWeights())


2022-02-17 12:13:23.222112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 12:13:23.226802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 12:13:23.227147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 12:13:23.227730: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/100
387/387 [==============================] - 1s 2ms/step - loss: 34.7730 - mean_absolute_percentage_error: 34.7730 - val_loss: 12.3213 - val_mean_absolute_percentage_error: 12.3213
Epoch 2/100
387/387 [==============================] - 1s 2ms/step - loss: 11.7749 - mean_absolute_percentage_error: 11.7749 - val_loss: 7.8143 - val_mean_absolute_percentage_error: 7.8143
Epoch 3/100
387/387 [==============================] - 1s 1ms/step - loss: 9.2416 - mean_absolute_percentage_error: 9.2416 - val_loss: 10.7593 - val_mean_absolute_percentage_error: 10.7593
Epoch 4/100
387/387 [==============================] - 1s 1ms/step - loss: 9.2056 - mean_absolute_percentage_error: 9.2056 - val_loss: 8.1576 - val_mean_absolute_percentage_error: 8.1576
Epoch 5/100
387/387 [==============================] - 1s 2ms/step - loss: 8.0269 - mean_absolute_percentage_error: 8.0269 - val_loss: 8.2438 - val_mean_absolute_percentage_error: 8.2438
Epoch 6/100
387/387 [==============================] - 1s

In [17]:

#Arrays to store mean and std of the change in weight per epoch data 
rms_weigth = np.zeros((epochs,6))
std_weigth = np.zeros((epochs,6))
for j,_ in enumerate(weights_dict.items()):
    if j%2 ==0 :
        key = list(weights_dict.keys())[j]
        print(key)
        for i in range(epochs):
            if i ==0: 
                rs_weigth = abs(((weights_dict[key])[:,:,1])-((weights_dict[key])[:,:,0]))
            else:
                rs_weigth = abs(((weights_dict[key])[:,:,i])-((weights_dict[key])[:,:,i-1]))
            rms_weigth[i,j//2] = np.mean(rs_weigth)
            std_weigth[i,j//2] = np.std(rs_weigth)
            
directory = '/home/sean/Documents/Work/Level 4/Level-4-Masters-Project/Batch_size_gradient_variance/Weights_Data'
extention = 'batch_size_{}'.format(batch_size)
joined = os.path.join(directory,extention)


np.savetxt(joined+'rms.csv',rms_weigth,delimiter = ',')
np.savetxt(joined+'std.csv',std_weigth,delimiter = ',')


w_1
w_2
w_3
w_4
w_5
w_6


[dict_keys(['w_1', 'b_1', 'w_2', 'b_2', 'w_3', 'b_3', 'w_4', 'b_4', 'w_5', 'b_5', 'w_6', 'b_6'])]

In [9]:
6//2

3